In [1]:
import pandas as pd
df=pd.read_csv('credit_data.csv')
df.head()

,id,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,1,no checking account,18,all credits at this bank paid back duly,car (used),1049,unknown/no savings account,< 1 yr,< 20,none,...,car or other,21,none,for free,1,skilled employee/official,0 to 2,no,no,good
1,2,no checking account,9,all credits at this bank paid back duly,others,2799,unknown/no savings account,1 <= ... < 4 yrs,25 <= ... < 35,none,...,unknown / no property,36,none,for free,2-3,skilled employee/official,3 or more,no,no,good
2,3,... < 0 DM,12,no credits taken/all credits paid back duly,retraining,841,... < 100 DM,4 <= ... < 7 yrs,25 <= ... < 35,none,...,unknown / no property,23,none,for free,1,unskilled - resident,0 to 2,no,no,good
3,4,no checking account,12,all credits at this bank paid back duly,others,2122,unknown/no savings account,1 <= ... < 4 yrs,20 <= ... < 25,none,...,unknown / no property,39,none,for free,2-3,unskilled - resident,3 or more,no,yes,good
4,5,no checking account,12,all credits at this bank paid back duly,others,2171,unknown/no savings account,1 <= ... < 4 yrs,< 20,none,...,car or other,38,bank,rent,2-3,unskilled - resident,0 to 2,no,yes,good


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       1000 non-null   int64 
 1   status                   1000 non-null   object
 2   duration                 1000 non-null   int64 
 3   credit_history           1000 non-null   object
 4   purpose                  1000 non-null   object
 5   amount                   1000 non-null   int64 
 6   savings                  1000 non-null   object
 7   employment_duration      1000 non-null   object
 8   installment_rate         1000 non-null   object
 9   other_debtors            1000 non-null   object
 10  present_residence        1000 non-null   object
 11  property                 1000 non-null   object
 12  age                      1000 non-null   int64 
 13  other_installment_plans  1000 non-null   object
 14  housing                  1000 non-null   

In [2]:
y=df['credit_risk']
x=df.drop(columns=['credit_risk','id'],axis=1)

In [7]:
cols = ['status','credit_history','purpose','savings','employment_duration','installment_rate','other_debtors','present_residence','property','other_installment_plans','housing','number_credits','job','people_liable','telephone','foreign_worker']
from sklearn.preprocessing import LabelEncoder
labels = LabelEncoder()
for col in cols:
    x[col]=labels.fit_transform(x[col])

In [8]:
x.head()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,other_debtors,present_residence,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker
0,3,18,0,2,1049,4,2,2,2,3,1,21,1,0,0,1,0,0,0
1,3,9,0,5,2799,4,0,1,2,0,3,36,1,0,1,1,1,0,0
2,0,12,4,8,841,0,1,1,2,3,3,23,1,0,0,3,0,0,0
3,3,12,0,5,2122,4,0,0,2,0,3,39,1,0,1,3,1,0,1
4,3,12,0,5,2171,4,0,2,2,3,1,38,0,2,1,3,0,0,1


In [12]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(
    x,
    y,
    stratify=y
)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

from sklearn.neighbors import KNeighborsClassifier

kn_model = KNeighborsClassifier(n_neighbors=3)
kn_model.fit(x_train_scaled,y_train)
kn_prediction = kn_model.predict(x_test_scaled)

In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_test,kn_prediction))

              precision    recall  f1-score   support

         bad       0.45      0.25      0.32        75
        good       0.73      0.87      0.79       175

    accuracy                           0.68       250
   macro avg       0.59      0.56      0.56       250
weighted avg       0.65      0.68      0.65       250



In [15]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
x_smote,y_smote = smote.fit_resample(x_train_scaled,y_train)
x_smote_train,x_smote_test,y_smote_train,y_smote_test = train_test_split(
    x_smote,
    y_smote,
    stratify=y_smote
)
smote_model = KNeighborsClassifier(n_neighbors=3)
smote_model.fit(x_smote_train,y_smote_train)
smote_prediction = smote_model.predict(x_smote_test)

In [16]:
print(classification_report(y_smote_test,smote_prediction))

              precision    recall  f1-score   support

         bad       0.75      0.80      0.77       132
        good       0.79      0.73      0.75       131

    accuracy                           0.76       263
   macro avg       0.77      0.76      0.76       263
weighted avg       0.77      0.76      0.76       263



In [17]:
from imblearn.under_sampling import RandomUnderSampler
rs=RandomUnderSampler(random_state=42)
x_rs,y_rs=rs.fit_resample(x_train_scaled,y_train)
x_rs_train,x_rs_test,y_rs_train,y_rs_test=train_test_split(
    x_rs,
    y_rs,
    stratify=y_rs
)

rs_model = KNeighborsClassifier(n_neighbors=3)
rs_model.fit(x_rs_train,y_rs_train)
rs_prediction = rs_model.predict(x_rs_test)

In [18]:
print(classification_report(y_rs_test,rs_prediction))

              precision    recall  f1-score   support

         bad       0.62      0.58      0.60        57
        good       0.60      0.64      0.62        56

    accuracy                           0.61       113
   macro avg       0.61      0.61      0.61       113
weighted avg       0.61      0.61      0.61       113



In [19]:
from sklearn.svm import SVC
svm=SVC(kernel='linear',random_state=42)
svm.fit(x_train,y_train)
prediction = svm.predict(x_test)
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

         bad       0.55      0.44      0.49        75
        good       0.78      0.85      0.81       175

    accuracy                           0.72       250
   macro avg       0.66      0.64      0.65       250
weighted avg       0.71      0.72      0.71       250



In [20]:
svm=SVC(kernel='linear',random_state=42)
svm.fit(x_train_scaled,y_train)
prediction = svm.predict(x_test_scaled)
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

         bad       0.64      0.09      0.16        75
        good       0.72      0.98      0.83       175

    accuracy                           0.71       250
   macro avg       0.68      0.54      0.49       250
weighted avg       0.69      0.71      0.63       250



In [21]:
svm=SVC(kernel='linear',random_state=42)
svm.fit(x_smote_train,y_smote_train)
prediction = svm.predict(x_smote_test)
print(classification_report(y_smote_test,prediction))

              precision    recall  f1-score   support

         bad       0.70      0.63      0.66       132
        good       0.66      0.73      0.70       131

    accuracy                           0.68       263
   macro avg       0.68      0.68      0.68       263
weighted avg       0.68      0.68      0.68       263



In [22]:
svm=SVC(kernel='linear',random_state=42)
svm.fit(x_rs_train,y_rs_train)
prediction = svm.predict(x_rs_test)
print(classification_report(y_rs_test,prediction))

              precision    recall  f1-score   support

         bad       0.67      0.70      0.68        57
        good       0.68      0.64      0.66        56

    accuracy                           0.67       113
   macro avg       0.67      0.67      0.67       113
weighted avg       0.67      0.67      0.67       113

